In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import cx_Oracle
from IPython.display import display
%matplotlib inline

#import warnings
#warnings.filterwarnings('ignore')

### Let's see the overview only with data coming from Z+ Shops 'active' customers (not bitiba and others)

In [2]:
# Data coming from all active customers in the last three years
data = pd.read_csv('comparison_eligibilities_per_site_active_customers_23082018.dsv', sep=';')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2847 entries, 0 to 2846
Data columns (total 6 columns):
OLD_TIER               2771 non-null float64
OLD_BOX                2771 non-null object
NEW_TIER               2763 non-null float64
NEW_BOX                2763 non-null object
K_REGISTRATION_S_ID    2847 non-null int64
AMOUNT                 2847 non-null int64
dtypes: float64(2), int64(2), object(2)
memory usage: 133.5+ KB


In [4]:
data.sample(5)

,OLD_TIER,OLD_BOX,NEW_TIER,NEW_BOX,K_REGISTRATION_S_ID,AMOUNT
165,5.0,B3,5.0,B3,4,34
2006,4.0,B2,4.0,B1,28,16
1125,4.0,B3,5.0,A,16,67
2034,4.0,A,4.0,B3,16,2
2822,4.0,B3,4.0,B1,3,1


In [5]:
# Site IDs from Zooplus Shops
np.sort(data['K_REGISTRATION_S_ID'].unique())[:26]

array([ 1,  2,  3,  4,  7,  8,  9, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22,
       23, 24, 25, 26, 28, 29, 30, 31, 32], dtype=int64)

In [6]:
# Get only data from Zooplus Shops
zp_data = data[data['K_REGISTRATION_S_ID'].isin(np.sort(data['K_REGISTRATION_S_ID'].unique())[:26])]

In [7]:
zp_data = zp_data[~zp_data.apply(lambda x: x.isnull().any(), axis=1)]

In [8]:
#Total amount of customers
zp_data['AMOUNT'].sum()

8350791

In [9]:
zp_crosstab = zp_data.fillna('not_segmented').groupby(['OLD_TIER', 'NEW_TIER'])['AMOUNT'].sum().unstack()

In [10]:
zp_crosstab_rel = zp_crosstab.apply(lambda x:x/x.sum() * 100, axis=1).fillna(0).round(2)

In [11]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [12]:
# Relative values of transition between old and new eligibility
zp_crosstab_rel.style.apply(highlight_max, axis=1)

NEW_TIER,0.0,4.0,5.0,6.0,24.0,25.0,1000.0,1004.0,1005.0
OLD_TIER,,,,,,,,,
0.0,75.29,20.41,0.65,2.44,0.48,0.02,0.15,0.56,0
4.0,14.21,52.04,23.98,8.96,0.01,0,0.46,0.29,0.05
5.0,18.01,12.49,56.4,12.83,0,0.03,0.1,0.08,0.06
6.0,63.25,0.06,0.05,36.64,0,0,0,0,0
24.0,9.26,0.32,0.12,14.28,58.48,17.54,0,0,0
25.0,6.67,0.14,0.48,20.95,12.85,58.92,0,0,0
1000.0,13.15,5.23,1.17,30.02,0,0,50.43,0,0
1004.0,18.76,2.32,0.31,0.71,0,0,0,72.45,5.45
1005.0,18.77,1.49,1.76,1.82,0,0,0,38.69,37.46


In [13]:
zp_crosstab['SUM_OLD_TIER'] = zp_crosstab.apply(np.sum, axis=1)

In [14]:
zp_crosstab.loc['SUM_NEW_TIER'] = zp_crosstab.apply(np.sum, axis=0)

In [15]:
#Calculate final % per Tier with the old/new routine

In [16]:
zp_crosstab['OLD_DIST%'] = zp_crosstab['SUM_OLD_TIER'] / zp_crosstab['SUM_OLD_TIER']['SUM_NEW_TIER'] * 100

In [17]:
zp_crosstab.loc['NEW_DIST%'] = zp_crosstab.loc['SUM_NEW_TIER'] / zp_crosstab.loc['SUM_NEW_TIER']['SUM_OLD_TIER'] * 100

In [18]:
# Absolute values of transition between old and new eligibility
zp_crosstab.fillna(0).astype(int)

NEW_TIER,0.0,4.0,5.0,6.0,24.0,25.0,1000.0,1004.0,1005.0,SUM_OLD_TIER,OLD_DIST%
OLD_TIER,,,,,,,,,,,
0.0,2109158,571855,18191,68259,13575,497,4285,15589,60,2801469,33
4.0,264934,970668,447218,167035,256,62,8487,5464,938,1865062,22
5.0,168038,116526,526160,119659,15,260,952,749,525,932884,11
6.0,1686739,1618,1235,977165,25,11,18,21,3,2666835,31
24.0,3418,118,46,5269,21579,6471,0,0,0,36901,0
25.0,884,19,64,2778,1704,7814,0,0,0,13263,0
1000.0,2431,968,216,5551,0,0,9325,0,0,18491,0
1004.0,2126,263,35,81,0,0,0,8212,618,11335,0
1005.0,854,68,80,83,0,0,0,1761,1705,4551,0


In [19]:
# Let's exclude new customers in the old and new routine (evaluated in reality by API calls from checkout and myzooplus)
zp_data = zp_data[(zp_data['OLD_TIER'] != float(6)) & (zp_data['NEW_TIER'] != float(6))]

In [20]:
#Exclude rows with NaN values (null)
zp_data = zp_data[~zp_data.apply(lambda x: x.isnull().any(), axis=1)]

In [21]:
# Total amount of customers after excluding Tier 6 customers in old or new routine
zp_data['AMOUNT'].sum()

5315241

In [22]:
# There is no Tier 6 now in our data
zp_data['OLD_TIER'].unique(), zp_data['NEW_TIER'].unique()

(array([   0.,    4.,    5.,   24., 1004., 1000.,   25., 1005.]),
 array([   0.,    5.,    4., 1004.,   24., 1000., 1005.,   25.]))

In [23]:
zp_crosstab = zp_data.fillna('not_segmented').groupby(['OLD_TIER', 'NEW_TIER'])['AMOUNT'].sum().unstack()

In [24]:
zp_crosstab_rel = zp_crosstab.apply(lambda x:x/x.sum() * 100, axis=1).fillna(0).round(2)

In [25]:
# Relative values of transition between old and new eligibility
zp_crosstab_rel.style.apply(highlight_max, axis=1)

NEW_TIER,0.0,4.0,5.0,24.0,25.0,1000.0,1004.0,1005.0
OLD_TIER,,,,,,,,
0.0,77.17,20.92,0.67,0.5,0.02,0.16,0.57,0
4.0,15.6,57.16,26.34,0.02,0,0.5,0.32,0.06
5.0,20.66,14.33,64.7,0,0.03,0.12,0.09,0.06
24.0,10.81,0.37,0.15,68.22,20.46,0,0,0
25.0,8.43,0.18,0.61,16.25,74.53,0,0,0
1000.0,18.79,7.48,1.67,0,0,72.06,0,0
1004.0,18.89,2.34,0.31,0,0,0,72.97,5.49
1005.0,19.11,1.52,1.79,0,0,0,39.41,38.16


In [26]:
zp_crosstab['SUM_OLD_TIER'] = zp_crosstab.apply(np.sum, axis=1)

In [27]:
zp_crosstab.loc['SUM_NEW_TIER'] = zp_crosstab.apply(np.sum, axis=0)

In [28]:
#Calculate final % per Tier with the old/new routine

In [29]:
zp_crosstab['OLD_DIST%'] = zp_crosstab['SUM_OLD_TIER'] / zp_crosstab['SUM_OLD_TIER']['SUM_NEW_TIER'] * 100

In [30]:
zp_crosstab.loc['NEW_DIST%'] = zp_crosstab.loc['SUM_NEW_TIER'] / zp_crosstab.loc['SUM_NEW_TIER']['SUM_OLD_TIER'] * 100

In [31]:
# Absolute values of transition between old and new eligibility
zp_crosstab.fillna(0).astype(int)

NEW_TIER,0.0,4.0,5.0,24.0,25.0,1000.0,1004.0,1005.0,SUM_OLD_TIER,OLD_DIST%
OLD_TIER,,,,,,,,,,
0.0,2109158,571855,18191,13575,497,4285,15589,60,2733210,51
4.0,264934,970668,447218,256,62,8487,5464,938,1698027,31
5.0,168038,116526,526160,15,260,952,749,525,813225,15
24.0,3418,118,46,21579,6471,0,0,0,31632,0
25.0,884,19,64,1704,7814,0,0,0,10485,0
1000.0,2431,968,216,0,0,9325,0,0,12940,0
1004.0,2126,263,35,0,0,0,8212,618,11254,0
1005.0,854,68,80,0,0,0,1761,1705,4468,0
SUM_NEW_TIER,2551843,1660485,992010,37129,15104,23049,31775,3846,5315241,100


In [33]:
np.round(2733210 / 8350791, 2),  np.round((5315241 - 2733210) / 8350791, 2)

(0.33, 0.31)

In [34]:
np.round(2551843 / 8350791, 2),  np.round((5315241 - 2551843) / 8350791, 2)

(0.31, 0.33)

In [32]:
for i in np.sort(data['K_REGISTRATION_S_ID'].unique())[:26]:
    print('                                                SITE ' + str(i) + '                                                           ')
    cus_data = zp_data[zp_data['K_REGISTRATION_S_ID'] == i]
    crosstab = cus_data.fillna('not_segmented').groupby(['OLD_TIER', 'NEW_TIER'])['AMOUNT'].sum().unstack()
    crosstab_rel = crosstab.apply(lambda x:x/x.sum() * 100, axis=1).fillna(0).round(2)
    display(crosstab_rel.style.apply(highlight_max, axis=1))
    crosstab['SUM_OLD_TIER'] = crosstab.apply(np.sum, axis=1)
    crosstab.loc['SUM_NEW_TIER'] = crosstab.apply(np.sum, axis=0)
    crosstab['OLD_DIST%'] = crosstab['SUM_OLD_TIER'] / crosstab['SUM_OLD_TIER']['SUM_NEW_TIER'] * 100
    crosstab.loc['NEW_DIST%'] = crosstab.loc['SUM_NEW_TIER'] / crosstab.loc['SUM_NEW_TIER']['SUM_OLD_TIER'] * 100
    display(crosstab.fillna(0).astype(int))
    print('Percentage of non-eligible customers with the old routine: ',
          np.round(cus_data[(cus_data['OLD_TIER'] == 0) | (cus_data['OLD_TIER'].isnull())]['AMOUNT'].sum() / cus_data['AMOUNT'].sum() * 100, 2))
    print('Percentage of non-eligible customers with the new routine: ',
          np.round(cus_data[(cus_data['NEW_TIER'] == 0) | (cus_data['NEW_TIER'].isnull())]['AMOUNT'].sum() / cus_data['AMOUNT'].sum() * 100, 2))
    print('_______________________________________________________________________________________________________________________')
    print('')

                                                SITE 1                                                           


NEW_TIER,0.0,4.0,5.0,24.0,25.0,1000.0,1004.0,1005.0
OLD_TIER,,,,,,,,
0.0,76.55,20.28,0.83,2.25,0.08,0.01,0,0
4.0,10.75,64.11,25.06,0.06,0.01,0.01,0,0
5.0,9.89,20.33,69.66,0.01,0.12,0,0,0
24.0,10.79,0.3,0.12,68.37,20.43,0,0,0
25.0,8.44,0.15,0.46,16.3,74.65,0,0,0
1000.0,12.5,8.33,4.17,0,0,75,0,0
1004.0,16.67,4.17,0,0,0,0,62.5,16.67
1005.0,0,0,0,0,0,0,85.71,14.29


NEW_TIER,0.0,4.0,5.0,24.0,25.0,1000.0,1004.0,1005.0,SUM_OLD_TIER,OLD_DIST%
OLD_TIER,,,,,,,,,,
0.0,461268,122184,4995,13538,495,31,24,0,602535,46
4.0,46014,274499,107320,246,57,28,19,2,428185,33
5.0,22179,45592,156209,14,258,2,2,2,224258,17
24.0,3393,93,37,21501,6425,0,0,0,31449,2
25.0,875,16,48,1690,7741,0,0,0,10370,0
1000.0,6,4,2,0,0,36,0,0,48,0
1004.0,4,1,0,0,0,0,15,4,24,0
1005.0,0,0,0,0,0,0,6,1,7,0
SUM_NEW_TIER,533739,442389,268611,36989,14976,97,66,9,1296876,100


Percentage of non-eligible customers with the old routine:  46.46
Percentage of non-eligible customers with the new routine:  41.16
_______________________________________________________________________________________________________________________

                                                SITE 2                                                           


NEW_TIER,0.0,4.0,5.0,24.0,25.0,1000.0,1004.0,1005.0
OLD_TIER,,,,,,,,
0.0,86.26,12.82,0.72,0.06,0,0.02,0.12,0
4.0,18.5,56.31,24.92,0.03,0.03,0.17,0.03,0
5.0,21.4,17.56,60.97,0,0,0,0.07,0
24.0,9.09,0,0,50,40.91,0,0,0
25.0,4.55,0,9.09,4.55,81.82,0,0,0
1000.0,0,9.09,18.18,0,0,72.73,0,0
1004.0,60,0,0,0,0,0,40,0
1005.0,0,0,0,0,0,0,33.33,66.67


...

NEW_TIER,0.0,4.0,5.0,SUM_OLD_TIER,OLD_DIST%
OLD_TIER,,,,,
0.0,7053,165,3,7221,52
4.0,502,3634,308,4444,32
5.0,204,111,1678,1993,14
SUM_NEW_TIER,7759,3910,1989,13658,100
NEW_DIST%,56,28,14,100,0


Percentage of non-eligible customers with the old routine:  52.87
Percentage of non-eligible customers with the new routine:  56.81
_______________________________________________________________________________________________________________________

                                                SITE 32                                                           


NEW_TIER,0.0,4.0,5.0
OLD_TIER,,,
0.0,99.28,0.72,0
4.0,38.77,57.46,3.78
5.0,4.11,26.03,69.86


NEW_TIER,0.0,4.0,5.0,SUM_OLD_TIER,OLD_DIST%
OLD_TIER,,,,,
0.0,2761,20,0,2781,81
4.0,195,289,19,503,14
5.0,6,38,102,146,4
SUM_NEW_TIER,2962,347,121,3430,100
NEW_DIST%,86,10,3,100,2


Percentage of non-eligible customers with the old routine:  81.08
Percentage of non-eligible customers with the new routine:  86.36
_______________________________________________________________________________________________________________________

